API

In [ ]:
#infos a scrap :
# • Name: The name of the restaurant.
# • Cuisine Type: Types of cuisine served (e.g., Italian, French, etc.).
# • Rating: The average user rating.
# • Opening Hours: Information about when the restaurant is open.
# • Location: The restaurant’s address or location details.
# • Average Price: The average cost of a meal.



In [7]:
!pip install requests

In [34]:
import pandas as pd

In [52]:
loc=pd.read_csv('data_in_progress/restaurants_tripadvisor_paris.csv')
#on garde que location_id
loc=loc['location_id']
loc.head(5)

0    25940733
1     1116702
2      775346
3    15707789
4     8297103
Name: location_id, dtype: int64

In [53]:
latlong=pd.read_csv('random_points_paris.csv',sep=',')
latlong.head()

,lat,long
0,48.835741,2.294406
1,48.875937,2.382459
2,48.869037,2.350171
3,48.851839,2.359734
4,48.852459,2.279369


In [54]:
import requests

import tqdm

location_id=[]
for i in tqdm.tqdm(range(len(latlong))):
    lat = latlong['lat'][i]
    long = latlong['long'][i]
    url = f"https://api.content.tripadvisor.com/api/v1/location/nearby_search?latLong={lat}%2C{long}&category=restaurants&language=en&key=7B630BA76BDC40F2BA71E1FBDB93F526"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    
    for i in response.json()['data']:
        location_id.append(i['location_id'])


    

100%|██████████| 200/200 [01:35<00:00,  2.08it/s]


In [55]:
print(location_id)

['17636607', '7682420', '12911007', '14895653', '6946218', '6856340', '25196628', '2563024', '28034388', '3389313', '12542749', '3619031', '8723793', '17840080', '23942096', '19997245', '4068654', '23881413', '3933272', '15012846', '17389083', '13374267', '23949307', '23127487', '24133554', '24133765', '13202382', '21380212', '26843310', '21345037', '5582194', '27901920', '783281', '26523446', '3436011', '8653135', '7809288', '21380211', '21167737', '26400958', '3347931', '2397841', '12658524', '17829218', '18904453', '21104307', '16746290', '786368', '1322113', '8377850', '1335814', '7130375', '13625754', '27095476', '22730871', '27842834', '27842889', '25080745', '1552207', '27375390', '12562326', '32788005', '7273105', '27356342', '24121560', '10339616', '17205859', '7202722', '27727186', '27304037', '28123337', '8774149', '2642068', '12439181', '27926533', '27266463', '5949410', '28016958', '844541', '1323224', '794837', '12492958', '2146860', '17424751', '27852390', '952073', '272

In [56]:
print(len(location_id))

2000


In [57]:
#on supprime les doublons
location_id = list(set(location_id))
print(len(location_id))


1788


In [59]:

#si il y a des elements de loc dans location_id on les supprime
location_id2=location_id

for i in loc:
    if i in location_id2:
        location_id2.remove(i)

print(len(location_id2))

1788


In [ ]:
#on a pas de doublons on ne supprime rien

In [60]:
#on recupere ce qu'on a besoin
# Initialiser une liste pour stocker les données
data_list = []
for i in tqdm.tqdm(range(len(location_id))):  # Parcourir les IDs des restaurants
    url = f"https://api.content.tripadvisor.com/api/v1/location/{location_id[i]}/details?language=fr&currency=EUR&key=7B630BA76BDC40F2BA71E1FBDB93F526"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)

    # Vérifier si la requête est réussie
    if response.status_code == 200:
        response_json = response.json()
        
        # Extraire les informations nécessaires
        data = {
            "location_id": response_json.get("location_id"),
            "name": response_json.get("name"),
            "description": response_json.get("description"),
            "adress_string": response_json.get("address_obj", {}).get("address_string"),
            "price_level": response_json.get("price_level"),
            "rating": response_json.get("rating"),
            "num_reviews": response_json.get("num_reviews"),
            "review_rating_count": response_json.get("review_rating_count"),
            "cuisine": ", ".join([c["localized_name"] for c in response_json.get("cuisine", [])]),
        }
        
        # Ajouter les données à la liste
        data_list.append(data)
    
    else:
        print(f"Erreur pour l'ID {location_id[i]} : {response.status_code}")
        continue  # Passer au suivant si erreur

# Créer un DataFrame avec les données collectées
df = pd.DataFrame(data_list)

# Afficher le DataFrame
print(df)

# (Optionnel) Enregistrer le DataFrame dans un fichier CSV
df.to_csv("data_in_progress/restaurants_tripadvisor_paris_2.csv", index=False, encoding="utf-8")

    
    


 25%|██▍       | 443/1788 [06:02<16:25,  1.36it/s]  

Erreur pour l'ID 3721618 : 429


100%|██████████| 1788/1788 [18:06<00:00,  1.65it/s]

     location_id                            name  \
0       27843529                 Schwartz's Deli   
1        9735299                           Plume   
2       27121628         Brunch Bande De Cheffes   
3       26364317                  The Englishman   
4       10547960                       Lavomatic   
...          ...                             ...   
1782    25102070            Restaurant Tt Nature   
1783    10192577     Le Monde Du Macaron Paris 5   
1784      792910                    le 20ème Art   
1785      718701  Restaurant La Mosquée de Paris   
1786      799446           Starbucks Victor Hugo   

                                            description  \
0                                                  None   
1                                                  None   
2     Traiteur solidaire pour évènements d'entrepris...   
3     À la recherche d’une soirée intime, à l’abri d...   
4                                                  None   
...                  

In [63]:
#on ouvre le fichier csv
df_resto2 = pd.read_csv("data_in_progress/restaurants_tripadvisor_paris_2.csv")
df_resto = pd.read_csv("data_in_progress/restaurants_tripadvisor_paris.csv")

In [64]:
#on concatene les deux dataframes
df_resto = pd.concat([df_resto, df_resto2], ignore_index=True)

In [65]:
#on creer un fichier csv restaurants_tripadvisor_paris_final.csv
df_resto.to_csv("restaurants_tripadvisor_paris_final.csv", index=False, encoding="utf-8")

In [66]:
#on affiche la longueur du dataframe
print(len(df_resto))

2159
